In [3]:
import pandas as pd

# Load the training data
train_data = pd.read_csv('Training_dt.csv')

# Drop 'Mask No' column if it exists
if 'Mask No' in train_data.columns:
    train_data = train_data.drop('Mask No', axis=1)

# Define categorical columns
categorical_cols = [
    'PRODUCT_NAME', 'Sub_purpose_code_based_on_risk', 'CRIB_SCORE', 'TOTAL INCOME',
    'Percentage_of_Total_Installments_to_Total_Current_Balance_slabs',
    'Percentage_of_Total_Current_Balance_to_Total_Amount_Granted_Limit_slabs',
    'Percentage_of_Total_Arrears_Amount_to_Total_Amount_Granted_Limit_slabs'
]

# Extract unique values for each categorical column
unique_categories = {col: train_data[col].dropna().unique().tolist() for col in categorical_cols}

# Save unique values to CSV (optional, for reference or future use)
unique_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in unique_categories.items()]))
unique_df.to_csv('unique_categories.csv', index=False)


In [1]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the saved files
ml_model = joblib.load('best_ml_model_v2.pkl')
preprocessor = joblib.load('preprocessor_v2.pkl')
label_encoder = joblib.load('label_encoder_v2.pkl')
ann_model = load_model('best_ann_model_v2.h5')

# Define unique categories
unique_df = pd.read_csv('unique_categories.csv')
unique_categories = {col: unique_df[col].dropna().tolist() for col in unique_df.columns}

def preprocess_input(inputs):
    # Convert inputs to DataFrame
    df = pd.DataFrame([inputs])
    
    # Apply preprocessing
    try:
        df_encoded = preprocessor.transform(df)
    except Exception as e:
        raise ValueError(f"Error during preprocessing: {e}")

    return df_encoded

def predict():
    try:
        # Collect user inputs
        inputs = {
            'PRODUCT_NAME': product_name_var.get(),
            'Sub_purpose_code_based_on_risk': sub_purpose_var.get(),
            'CRIB_SCORE': crib_score_var.get(),
            'TOTAL INCOME': total_income_var.get(),
            'Percentage_of_Total_Installments_to_Total_Current_Balance_slabs': balance_installments_var.get(),
            'Percentage_of_Total_Current_Balance_to_Total_Amount_Granted_Limit_slabs': balance_limit_var.get(),
            'Percentage_of_Total_Arrears_Amount_to_Total_Amount_Granted_Limit_slabs': arrears_limit_var.get(),
            'LEASE_TENOR_INCLUDING_HP': lease_tenor_entry.get(),
            'CUSTOMER AGE': customer_age_entry.get(),
            'Exp': exp_entry.get(),
            'YOM': yom_entry.get()
        }

        # Convert numerical inputs to float
        numerical_keys = ['LEASE_TENOR_INCLUDING_HP', 'CUSTOMER AGE', 'Exp', 'YOM']
        for key in numerical_keys:
            try:
                inputs[key] = float(inputs[key])
            except ValueError:
                result_label.config(text="Invalid input for numerical values.")
                return

        # Preprocess the data
        X_processed = preprocess_input(inputs)

        # Get predictions
        ml_predictions = ml_model.predict_proba(X_processed)
        X_combined = np.hstack([X_processed, ml_predictions])
        ann_predictions = ann_model.predict(X_combined)

        # Use decision threshold to classify
        threshold = 0.40
        y_pred = (ann_predictions[:, 1] >= threshold).astype(int)
        y_pred_label = label_encoder.inverse_transform(y_pred)

        # Display the result
        result_label.config(text=f'Predicted Cluster: {y_pred_label[0]}')

    except Exception as e:
        result_label.config(text=f"Error: {e}")

# Create the GUI
root = tk.Tk()
root.title("Model Prediction GUI")

# Create and place widgets for numerical input
tk.Label(root, text="LEASE_TENOR_INCLUDING_HP").grid(row=0, column=0)
lease_tenor_entry = tk.Entry(root)
lease_tenor_entry.grid(row=0, column=1)

tk.Label(root, text="CUSTOMER AGE").grid(row=1, column=0)
customer_age_entry = tk.Entry(root)
customer_age_entry.grid(row=1, column=1)

tk.Label(root, text="Exp").grid(row=2, column=0)
exp_entry = tk.Entry(root)
exp_entry.grid(row=2, column=1)

tk.Label(root, text="YOM").grid(row=3, column=0)
yom_entry = tk.Entry(root)
yom_entry.grid(row=3, column=1)

# Create and place dropdown menus for categorical input
tk.Label(root, text="PRODUCT_NAME").grid(row=4, column=0)
product_name_var = tk.StringVar()
product_name_menu = ttk.Combobox(root, textvariable=product_name_var, values=unique_categories['PRODUCT_NAME'])
product_name_menu.grid(row=4, column=1)

tk.Label(root, text="Sub_purpose_code_based_on_risk").grid(row=5, column=0)
sub_purpose_var = tk.StringVar()
sub_purpose_menu = ttk.Combobox(root, textvariable=sub_purpose_var, values=unique_categories['Sub_purpose_code_based_on_risk'])
sub_purpose_menu.grid(row=5, column=1)

tk.Label(root, text="CRIB_SCORE").grid(row=6, column=0)
crib_score_var = tk.StringVar()
crib_score_menu = ttk.Combobox(root, textvariable=crib_score_var, values=unique_categories['CRIB_SCORE'])
crib_score_menu.grid(row=6, column=1)

tk.Label(root, text="TOTAL INCOME").grid(row=7, column=0)
total_income_var = tk.StringVar()
total_income_menu = ttk.Combobox(root, textvariable=total_income_var, values=unique_categories['TOTAL INCOME'])
total_income_menu.grid(row=7, column=1)

tk.Label(root, text="Percentage_of_Total_Installments_to_Total_Current_Balance_slabs").grid(row=8, column=0)
balance_installments_var = tk.StringVar()
balance_installments_menu = ttk.Combobox(root, textvariable=balance_installments_var, values=unique_categories['Percentage_of_Total_Installments_to_Total_Current_Balance_slabs'])
balance_installments_menu.grid(row=8, column=1)

tk.Label(root, text="Percentage_of_Total_Current_Balance_to_Total_Amount_Granted_Limit_slabs").grid(row=9, column=0)
balance_limit_var = tk.StringVar()
balance_limit_menu = ttk.Combobox(root, textvariable=balance_limit_var, values=unique_categories['Percentage_of_Total_Current_Balance_to_Total_Amount_Granted_Limit_slabs'])
balance_limit_menu.grid(row=9, column=1)

tk.Label(root, text="Percentage_of_Total_Arrears_Amount_to_Total_Amount_Granted_Limit_slabs").grid(row=10, column=0)
arrears_limit_var = tk.StringVar()
arrears_limit_menu = ttk.Combobox(root, textvariable=arrears_limit_var, values=unique_categories['Percentage_of_Total_Arrears_Amount_to_Total_Amount_Granted_Limit_slabs'])
arrears_limit_menu.grid(row=10, column=1)

# Predict button
predict_button = tk.Button(root, text="Predict", command=predict)
predict_button.grid(row=11, column=0, columnspan=2)

# Label to display results
result_label = tk.Label(root, text="Predicted Cluster: ")
result_label.grid(row=12, column=0, columnspan=2)

# Start the GUI event loop
root.mainloop()


## ----------------------------------------NEW_final------------------------------------

In [2]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import numpy as np
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from datetime import datetime

# Load the saved files
ml_model = joblib.load('best_ml_model_v2.pkl')
preprocessor = joblib.load('preprocessor_v2.pkl')
label_encoder = joblib.load('label_encoder_v2.pkl')
ann_model = load_model('best_ann_model_v2.h5')

# Define unique categories
unique_df = pd.read_csv('unique_categories.csv')
unique_categories = {col: unique_df[col].dropna().tolist() for col in unique_df.columns}

def preprocess_input(inputs):
    # Convert inputs to DataFrame
    df = pd.DataFrame([inputs])
    
    # Apply preprocessing
    try:
        df_encoded = preprocessor.transform(df)
    except Exception as e:
        raise ValueError(f"Error during preprocessing: {e}")

    return df_encoded

def validate_inputs(inputs):
    # Check the validations
    if inputs['CRIB_SCORE'] == 'below 0':
        return 'Red'
    if inputs['CUSTOMER AGE'] > 70:
        return 'Red'
    if inputs['Exp'] > 0.8:
        return 'Red'
    if inputs['YOM'] > datetime.now().year:
        return 'Red'
    return None

def predict():
    try:
        # Collect user inputs
        inputs = {
            'PRODUCT_NAME': product_name_var.get(),
            'Sub_purpose_code_based_on_risk': sub_purpose_var.get(),
            'CRIB_SCORE': crib_score_var.get(),
            'TOTAL INCOME': total_income_var.get(),
            'Percentage_of_Total_Installments_to_Total_Current_Balance_slabs': balance_installments_var.get(),
            'Percentage_of_Total_Current_Balance_to_Total_Amount_Granted_Limit_slabs': balance_limit_var.get(),
            'Percentage_of_Total_Arrears_Amount_to_Total_Amount_Granted_Limit_slabs': arrears_limit_var.get(),
            'LEASE_TENOR_INCLUDING_HP': lease_tenor_entry.get(),
            'CUSTOMER AGE': customer_age_entry.get(),
            'Exp': exp_entry.get(),
            'YOM': yom_entry.get()
        }

        # Convert numerical inputs to float
        numerical_keys = ['LEASE_TENOR_INCLUDING_HP', 'CUSTOMER AGE', 'Exp', 'YOM']
        for key in numerical_keys:
            try:
                inputs[key] = float(inputs[key])
            except ValueError:
                result_label.config(text="Invalid input for numerical values.")
                return

        # Validate the inputs
        validation_result = validate_inputs(inputs)
        if validation_result:
            result_label.config(text=f'Predicted Cluster: {validation_result}')
            return

        # Preprocess the data
        X_processed = preprocess_input(inputs)

        # Get predictions
        ml_predictions = ml_model.predict_proba(X_processed)
        X_combined = np.hstack([X_processed, ml_predictions])
        ann_predictions = ann_model.predict(X_combined)

        # Use decision threshold to classify
        threshold = 0.40
        y_pred = (ann_predictions[:, 1] >= threshold).astype(int)
        y_pred_label = label_encoder.inverse_transform(y_pred)

        # Display the result
        result_label.config(text=f'Predicted Cluster: {y_pred_label[0]}')

    except Exception as e:
        result_label.config(text=f"Error: {e}")

# Create the GUI
root = tk.Tk()
root.title("Model Prediction GUI")

# Create and place widgets for numerical input
tk.Label(root, text="LEASE_TENOR_INCLUDING_HP").grid(row=0, column=0)
lease_tenor_entry = tk.Entry(root)
lease_tenor_entry.grid(row=0, column=1)

tk.Label(root, text="CUSTOMER AGE").grid(row=1, column=0)
customer_age_entry = tk.Entry(root)
customer_age_entry.grid(row=1, column=1)

tk.Label(root, text="Exp").grid(row=2, column=0)
exp_entry = tk.Entry(root)
exp_entry.grid(row=2, column=1)

tk.Label(root, text="YOM").grid(row=3, column=0)
yom_entry = tk.Entry(root)
yom_entry.grid(row=3, column=1)

# Create and place dropdown menus for categorical input
tk.Label(root, text="PRODUCT_NAME").grid(row=4, column=0)
product_name_var = tk.StringVar()
product_name_menu = ttk.Combobox(root, textvariable=product_name_var, values=unique_categories['PRODUCT_NAME'])
product_name_menu.grid(row=4, column=1)

tk.Label(root, text="Sub_purpose_code_based_on_risk").grid(row=5, column=0)
sub_purpose_var = tk.StringVar()
sub_purpose_menu = ttk.Combobox(root, textvariable=sub_purpose_var, values=unique_categories['Sub_purpose_code_based_on_risk'])
sub_purpose_menu.grid(row=5, column=1)

tk.Label(root, text="CRIB_SCORE").grid(row=6, column=0)
crib_score_var = tk.StringVar()
crib_score_menu = ttk.Combobox(root, textvariable=crib_score_var, values=unique_categories['CRIB_SCORE'])
crib_score_menu.grid(row=6, column=1)

tk.Label(root, text="TOTAL INCOME").grid(row=7, column=0)
total_income_var = tk.StringVar()
total_income_menu = ttk.Combobox(root, textvariable=total_income_var, values=unique_categories['TOTAL INCOME'])
total_income_menu.grid(row=7, column=1)

tk.Label(root, text="Percentage_of_Total_Installments_to_Total_Current_Balance_slabs").grid(row=8, column=0)
balance_installments_var = tk.StringVar()
balance_installments_menu = ttk.Combobox(root, textvariable=balance_installments_var, values=unique_categories['Percentage_of_Total_Installments_to_Total_Current_Balance_slabs'])
balance_installments_menu.grid(row=8, column=1)

tk.Label(root, text="Percentage_of_Total_Current_Balance_to_Total_Amount_Granted_Limit_slabs").grid(row=9, column=0)
balance_limit_var = tk.StringVar()
balance_limit_menu = ttk.Combobox(root, textvariable=balance_limit_var, values=unique_categories['Percentage_of_Total_Current_Balance_to_Total_Amount_Granted_Limit_slabs'])
balance_limit_menu.grid(row=9, column=1)

tk.Label(root, text="Percentage_of_Total_Arrears_Amount_to_Total_Amount_Granted_Limit_slabs").grid(row=10, column=0)
arrears_limit_var = tk.StringVar()
arrears_limit_menu = ttk.Combobox(root, textvariable=arrears_limit_var, values=unique_categories['Percentage_of_Total_Arrears_Amount_to_Total_Amount_Granted_Limit_slabs'])
arrears_limit_menu.grid(row=10, column=1)

# Predict button
predict_button = tk.Button(root, text="Predict", command=predict)
predict_button.grid(row=11, column=0, columnspan=2)

# Label to display results
result_label = tk.Label(root, text="Predicted Cluster: ")
result_label.grid(row=12, column=0, columnspan=2)

# Start the GUI event loop
root.mainloop()
